In [4]:
ls ../functions

basal_model_calcs.py            interface_GAMS.py
conversion_equations.py         mRNA_ratios.py
create_cAct_cInh_vals.py        parameter_optimization.py
create_data_for_single_gene.py  promoter_solving_core.py
GA_core.py                      __pycache__/


In [5]:
# imports and loadings
import os
import sys
sys.path.insert(0, '../functions/')
import interface_GAMS as iG
import conversion_equations as ce
import pandas as pd
import numpy as np
import shutil

# settings
testing = True
small_dataset = True # only use the first 50 samples, mostly useful when using a limited license
pull_best_paras = False # pull best results from parameter_optimization, else use defaults


# load in settings flags
settings_df = pd.read_csv('../options/settings.csv', index_col = 0)
flags_filepath = settings_df.loc['gene_flags_filepath']['Setting']
TF_flags_filepath = settings_df.loc['TF_flags_filepath']['Setting']
GAMS_exec = str(settings_df.loc['GAMS_executable_path']['Setting'])
if GAMS_exec == 'nan':
    GAMS_exec = 'gams'
    
# load in flags dataframes
flags_df = pd.read_csv(flags_filepath, index_col = 0)
flags_df = flags_df[flags_df['include'] == True]
TF_flags_df = pd.read_csv(TF_flags_filepath, index_col = 0)

# below are the default flags used if nothing is pre-set
# set flags by editing the "saved_flags.csv" in the ../data folder
t_half_life_deg = 300
stable_flags = { # these do not change gene by gene
    # overall
    'only_create_ratios' : False,
    'only_check_KdRNAPAct' : False, # if True, quit out of code after generating KdRNAPAct, done to see if it is generating valid values through sanity check plots
    'include_Amy_samples' : True, # append on Amy's stationary phase samples to analysis
    'remove_outliers' : True, # removes samples that do not correlate well with others, see ../data_cleaning/1_locate_outliers_to_drop.ipynb
    'drop_basal_conds' : True, # if True, removes basal conditions from sample after they're used to calculate ratios (useful when their outliers)
    
    # KdRNAPAct optimization
    'KdRNAPAct_sanity' : True, # if True, return sanity plots from this optimization
    
    # GAMs
    'limit_TF_conc_by_actual' : False, # limits the TF concentrations for the model by the actual values, otherwise lets it be a very wide range
    'supress_output' : False,
    'use_greedy' : True, # use the greedy algo values (if False, uses the results of the GA)
    'run_on_all' : False, # run on all genes that are in the saved output folder
    'limit_samples' : flags_df.index.to_list(), # if run_on_all is False, limit to these samples (or which of them are available)
    'delete_old' : True,
    'run_seperate' : False, # run cActivator and cInhibitor solvers seperately
    
    # input constants for GAMs (all get logged inside GAMs so pass in un-logged)
    'act_TF_conc_lo' : 2.902870141566294e-13 / 1000000000, # minimum TF conc found in Heineman data
    'act_TF_conc_up' : 0.00014190659526601638 * 1000000000, # max of ^
    'act_Kd_lo' : 11e-9 / 1000000000, # 11 - 35 nM (1e-9) is the answer here - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4646316/
    'act_Kd_up' : 35e-9 * 1000000000, # from above
    'inh_TF_conc_lo' : 2.902870141566294e-13 / 1000000000, # minimum TF conc found in Heineman data
    'inh_TF_conc_up' : 0.00014190659526601638 * 1000000000, # max of ^
    'inh_Kd_lo' : 11e-9 / 1000000000, # 11 - 35 nM (1e-9) is the answer here - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4646316/
    'inh_Kd_up' : 35e-9 * 1000000000, # from above
    'inh_metab_Total_lo' : 0.000038 / 1000000000, # minimum of arginine concentration in stationary phase samples, div a buffer
    'inh_metab_Total_up' : 0.000408 * 1000000000, # maximum of arginine concentration in stationary phase samples, mult a buffer
    'act_metab_Total_lo' : 0.000038 / 1000000000, # minimum of arginine concentration in stationary phase samples, div a buffer
    'act_metab_Total_up' : 0.000408 * 1000000000, # maximum of arginine concentration in stationary phase samples, mult a buffer
    
    # best for argR
    #'act_TF_conc_lo' : 2.902870141566294e-13 / 100, # minimum TF conc found in Heineman data
    #'act_TF_conc_up' : 0.00014190659526601638 * 100, # max of ^
    #'act_Kd_lo' : 11e-9 / 100, # 11 - 35 nM (1e-9) is the answer here - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4646316/
    #'act_Kd_up' : 35e-9 * 100, # from above
    #'inh_TF_conc_lo' : 2.902870141566294e-13 / 100, # minimum TF conc found in Heineman data
    #'inh_TF_conc_up' : 0.00014190659526601638 * 100, # max of ^
    #'inh_Kd_lo' : 11e-9 / 100, # 11 - 35 nM (1e-9) is the answer here - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4646316/
    #'inh_Kd_up' : 35e-9 * 100, # from above
    #'metab_Total_lo' : 0.000038 / 100, # minimum of arginine concentration in stationary phase samples, div a buffer
    #'metab_Total_up' : 0.000408 * 100, # maximum of arginine concentration in stationary phase samples, mult a buffer
    
    
    # objective function weightings
    'weight_act_obj1' : 1,
    'weight_inh_obj1' : 1,
    'weight_act_obj2' : 0,
    'weight_inh_obj2' : 0,
    'weight_mRNA_match' : 1.0001,
    'weight_act_corr' : 0.00000000000000001,
    'weight_inh_corr' : 0.00000000000000001,
    
    
    # misc
    'eq_str' : 'Eq(mRNARatio,((cActivator*KdRNAP + KdRNAPAct)*(KdRNAP + RNAP + \
            KeqOpening*RNAP))/((1 + cActivator + cInhibitor)*KdRNAP*KdRNAPAct + \
            cActivator*KdRNAP*(1 + KeqOpening)*RNAP + KdRNAPAct*(1 + \
            KeqOpening)*RNAP))',
    
    # cell_constants'
    'cell_constants_RNAP': 10**-6,
    'cell_constants_mRNA_total': 1800, # Total mRNA/cell from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3554401
    'cell_constants_cell_volume': 10**-15, # Liters from https://bionumbers.hms.harvard.edu/bionumber.aspx?id=100004&ver=19
    'cell_constants_kDeg': np.log(2)/t_half_life_deg, # Rate of degradation
    'cell_constants_promoterConcVal': 10**-9, # Promoter concentration
    'cell_constants_u': 1/3600, # Growth rate
}

# pick best parameters from the optimization results
best_paras = None
if pull_best_paras:
    # not set up at all right now for multi-iM, unclear what this will look like
    pickle_in = open('../data/interim/misc_dictionaries/case_to_best_paras.pkl', 'rb')
    case_to_best_paras = pickle.load(pickle_in)
    pickle_in.close()
    best_paras = dict(case_to_best_paras[case]['Value'])
    
def show_figure(fig):

    # create a dummy figure and use its
    # manager to display "fig"

    dummy = plt.figure()
    new_manager = dummy.canvas.manager
    new_manager.canvas.figure = fig
    fig.set_canvas(new_manager.canvas)

# run and read GAMS

In [2]:
# run GAMs
cell_constants = {}
cell_constants.update({
    'RNAP' : stable_flags['cell_constants_RNAP'],
    'mRNA_total' : stable_flags['cell_constants_mRNA_total'],
    'cell_volume' : stable_flags['cell_constants_cell_volume'],
    'kDeg' : stable_flags['cell_constants_kDeg'],
    'promoterConcVal' : stable_flags['cell_constants_promoterConcVal'],
})

# run through each act_inh combination, run GAMS on each
combos = set()
for index, row in flags_df.iterrows():
    combos.add((str(row['act_iM']), str(row['inh_iM'])))
for (act_iM, inh_iM) in combos:
    GAMs_run_dir = '../GAMS/runs/'+(act_iM+'__'+inh_iM).replace(' ', '_').replace('/', '_')
    if os.path.exists(GAMs_run_dir):
        shutil.rmtree(GAMs_run_dir)
    os.mkdir(GAMs_run_dir)
    os.mkdir(GAMs_run_dir+'/input_files')
    os.mkdir(GAMs_run_dir+'/output_files')
    os.mkdir(GAMs_run_dir+'/input_GDX')
    os.mkdir(GAMs_run_dir+'/output_GDX')
    print('Running at '+GAMs_run_dir)
    keep = []
    for index, row in flags_df.iterrows():
        if str(row['act_iM']) == act_iM and str(row['inh_iM']) == inh_iM:
            keep.append(index)
    bby_flags_df = flags_df.loc[keep]
    bby_TF_flags_df = TF_flags_df.loc[list(set([act_iM, inh_iM]).intersection(TF_flags_df.index))]
    stable_flags.update({'limit_samples' : bby_flags_df.index.to_list()})
    stable_flags.update({'small_dataset' : small_dataset})
    iG.run_multi_GAMS(bby_flags_df, bby_TF_flags_df, stable_flags, cell_constants, GAMs_run_dir, GAMS_exec = GAMS_exec, parameter_flags = best_paras)

Running at ../GAMS/runs/Fatty_Acid__nan
--- Job combined_model Start 08/01/24 20:50:22 44.1.1 27c4d1f8 LEX-LEG x86 64bit/Linux
--- Applying:
    /opt/gams/gams44.1_linux_x64_64_sfx/gmsprmun.txt
--- GAMS Parameters defined
    Input /home/chris/github/regulonML/GAMS/runs/Fatty_Acid__nan/combined_model.gms
    ScrDir /home/chris/github/regulonML/GAMS/runs/Fatty_Acid__nan/225a/
    SysDir /opt/gams/gams44.1_linux_x64_64_sfx/
Licensee: GAMS Community License for Christopher DalldorfG230829|0002AO-GEN
          University of California, San Diego, Bioengineering        CL6627
          /opt/gams/gams44.1_linux_x64_64_sfx/gamslice.txt
          Christopher Dalldorf, cdalldor@eng.ucsd.edu                      
          Community license for demonstration and instructional purposes only
Processor information: 2 socket(s), 12 core(s), and 24 thread(s) available
GAMS 44.1.1   Copyright (C) 1987-2023 GAMS Development. All rights reserved
--- Starting compilation
--- combined_model.gms(65) 3 Mb
-

     26   4        8.6331134541E+07 9.1E+06    70 1.0E-01    6 F  T
     31   4        8.3983781942E+07 2.7E+06    67 3.5E-01    4 F  T
     36   4        7.6604434190E+07 5.1E+06    70 1.0E+00    3 F  F
     41   4        7.0886359388E+07 2.9E+06    64 2.3E+00    2 F  T
     46   4        7.0500521465E+07 8.9E+05    63 1.8E-01   16 F  T
     51   4        6.9517968484E+07 1.7E+06    57 1.0E-01   10 F  T
     56   4        6.9325915637E+07 7.2E+04    52 1.0E+00    3 F  T
 
   Iter Phase Ninf     Objective     RGmax    NSB   Step InItr MX OK
     61   4        6.9204428949E+07 1.7E+05    61 1.0E+00    2 F  T
     66   4        2.8553888036E+07 6.1E+04    60 1.0E+00    3 T  T
     71   4        2.5643281678E+06 5.8E+06    53 1.0E+00    6 F  T
     76   4        0.0000000000E+00 1.3E+04    52 0.0E+00    1 F  F
 
 ** Feasible solution. The tolerances are minimal and
    there is no change in objective although the reduced
    gradient is greater than the tolerance.
 
--- Reading solution f

     31   4        3.2837191481E+05 2.7E+03    36 5.2E+00    2 F  T
     36   4        3.1183018917E+05 2.4E+02    37 1.0E-01    6 F  T
     41   4        2.6397464405E+05 8.3E+04    31 1.0E-01    4 F  T
     46   4        2.0363221526E+05 3.1E+04    34 1.0E-01    3 F  T
     51   4        1.9001098727E+05 4.9E+03    31 1.0E+00    5 F  T
     56   4        1.7389229685E+05 7.3E+04    36 4.7E-01    4 F  T
 
   Iter Phase Ninf     Objective     RGmax    NSB   Step InItr MX OK
     61   4        9.9777418132E+04 1.3E+05    35 1.0E+00    3 F  T
     66   4        8.1808244631E+04 2.4E+04    36 1.0E+00    4 F  T
     71   4        7.2023098347E+04 5.1E+03    36 1.0E+00    6 F  T
     76   4        7.0987533101E+04 4.2E+01    36 1.0E+00    2 F  T
     81   4        7.0588522531E+04 2.3E+03    36 3.9E-01   20 F  T
     86   4        7.0554927118E+04 4.0E+01    37 1.0E+00    2 F  T
     91   4        7.0204350994E+04 7.8E+00    37 2.2E-01    6 F  T
     96   4        7.0080603312E+04 5.7E+01  

# correlation matrices

In [6]:
# create correlation matrices, used for some plotting later

# merge together log_tpm_df files
log_tpm_df = pd.read_csv('../data/external/imodulon_info/log_tpm.csv', index_col = 0)
starve_log_tpm = pd.read_csv('../data/external/validation_data_sets/stationary_phase/cleaned_log_tpm_qc.csv', index_col = 0)
to_blank_inds = list(set(log_tpm_df.index) - set(starve_log_tpm.index))
# need to create zero rows for missing values
zeros_data = {col : 0 for col in starve_log_tpm.columns}
zeros_df = pd.DataFrame(zeros_data, index = to_blank_inds)
starve_log_tpm = pd.concat([starve_log_tpm, zeros_df])
starve_log_tpm = starve_log_tpm.loc[log_tpm_df.index]
log_tpm_df = pd.concat([starve_log_tpm, log_tpm_df], axis = 1)

cases = set([(row['act_iM'], row['inh_iM']) for _, row in flags_df.iterrows()])
collect_actuals = []
collect_predictions = []
for iMs_run in cases:
    clean = [x for x in iMs_run if str(x) != 'nan']
    case = '__'.join([str(iM) for iM in iMs_run]).replace(' ', '_').replace('/', '_')
    if not os.path.exists('../GAMS/runs/'+case):
        continue
    
    # let's find our run_dir
    try:
        GAMs_run_dir = '../GAMS/runs/'+case
        mRNA_df, GAMS_calc_cAct, cAct_kd_df, act_metab_df, act_kd_metab_df, GAMS_calc_cInh, cInh_kd_df, inh_metab_df, inh_kd_metab_df = iG.read_multi_GAMs(GAMs_run_dir)
    except:
        continue # Usually means it hasn't been run yet
    actual_ratio = ce.log_tpm_df_to_mRNA_ratio_df(log_tpm_df.loc[mRNA_df.columns], flags_df.loc[mRNA_df.columns])

    collect_actuals.append(actual_ratio)
    collect_predictions.append(mRNA_df)

merge_actual = pd.concat(collect_actuals, axis = 1)
merge_pred = pd.concat(collect_predictions, axis = 1)
overlap = list(set(merge_actual.index).intersection(set(merge_pred.index)))
actual_mRNA_df = merge_actual.loc[:,~merge_actual.columns.duplicated()].astype(float)
pred_mRNA_df = merge_pred.loc[:,~merge_pred.columns.duplicated()].astype(float)

# create sample-sample and gene-gene correlation dataframe
sample_corr_df = pd.DataFrame(index = actual_mRNA_df.index, columns = actual_mRNA_df.index)
for sample1 in pred_mRNA_df.index:
    for sample2 in pred_mRNA_df.index:
        sample_corr_df.at[sample1, sample2] = actual_mRNA_df.loc[sample1].corr(pred_mRNA_df.loc[sample2])
gene_corr_df = pd.DataFrame(index = actual_mRNA_df.columns, columns = actual_mRNA_df.columns)
for sample1 in pred_mRNA_df.columns:
    for sample2 in pred_mRNA_df.columns:
        gene_corr_df.at[sample1, sample2] = actual_mRNA_df[sample1].corr(pred_mRNA_df[sample2])

# these things take a while to run, let's save them off
sample_corr_df.to_csv('../data/interim/samples_correlation_df.csv')
gene_corr_df.to_csv('../data/interim/genes_correlation_df.csv')